In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [5]:
#pip install category_encoders

In [60]:
#pip install XGboost

In [2]:
print(pd.__version__)
print(np.__version__)

1.3.5
1.21.5


In [4]:
#pip install --upgrade pandas

In [3]:
#pip install --upgrade numpy

In [3]:
train_df = 'data/train.csv'
test_df = 'data/test.csv'
train_df = pd.read_csv(train_df)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
train_df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


#Handling Null values

In [6]:
objects = train_df.select_dtypes('object')
objects

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [7]:
train_df.isnull().sum().sum()

6965

In [8]:
precent_null = (train_df.isnull().sum().sum()/(train_df.shape[0]*train_df.shape[1]))*100
print('Percentage of null values: ' + str(precent_null) )

Percentage of null values: 5.889565364451209


In [9]:
s = (train_df.isnull().sum() > 0)
null_columns = s[s].index
null_columns

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [10]:
train_df[null_columns].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

________________________________________________________________________________
Filling up NA values
________________________________________________________________________________

In [11]:
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(0)
print(train_df['LotFrontage'].isnull().sum())

0


In [12]:
train_df['Alley'].fillna(0,inplace=True)
print(train_df['Alley'].isnull().sum())

0


In [13]:
train_df['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [14]:
train_df['MasVnrType'].fillna('None',inplace=True)
print(train_df['MasVnrType'].isnull().sum())

0


In [15]:
train_df['MasVnrArea'].fillna(0,inplace=True)
print(train_df['MasVnrArea'].isnull().sum())

0


In [16]:
train_df[train_df['BsmtQual'].isnull() & train_df['BsmtQual'].notnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [17]:
basement_null_index = train_df[train_df['BsmtQual'].isnull()].index
basement_null_index

Int64Index([  17,   39,   90,  102,  156,  182,  259,  342,  362,  371,  392,
             520,  532,  533,  553,  646,  705,  736,  749,  778,  868,  894,
             897,  984, 1000, 1011, 1035, 1045, 1048, 1049, 1090, 1179, 1216,
            1218, 1232, 1321, 1412],
           dtype='int64')

In [18]:
basement_cols = ['BsmtQual','BsmtCond', 'BsmtExposure' , 'BsmtFinType1' , 'BsmtFinType2']
train_df[basement_cols].iloc[basement_null_index]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
17,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN


In [19]:
#All of the basement columns are null and fill them with equivalent 0 or None or NA value/category

train_df['BsmtQual'].fillna('NA',inplace=True)
train_df['BsmtCond'].fillna('NA',inplace=True)
train_df['BsmtExposure'].fillna('NA',inplace=True)
train_df['BsmtFinType1'].fillna('NA',inplace=True)
train_df['BsmtFinType2'].fillna('NA',inplace=True)
train_df['BsmtFinType2'].isna().any().sum()


0

In [20]:
train_df[basement_cols].any().fillna('NA',inplace=True)
train_df[basement_cols].isna().sum()

BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
dtype: int64

In [21]:
train_df[train_df['Electrical'].isnull()].index
train_df.iloc[train_df[train_df['Electrical'].isnull()].index]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,1380,80,RL,73.0,9735,Pave,0,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,167500


In [22]:
train_df['Electrical'].fillna('SBrkr',inplace=True)

In [23]:
train_df['FireplaceQu'].fillna('NA',inplace=True)

In [24]:
garage_columns = ['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond' ]
garage_null_cols = train_df[train_df['GarageType'].isnull()].index
train_df[garage_columns].iloc[garage_null_cols].notna().any().sum()

#all these columns are null

0

In [25]:
train_df['GarageType'].fillna('NA',inplace=True)
train_df['GarageYrBlt'].fillna('NA',inplace=True)
train_df['GarageFinish'].fillna('NA',inplace=True)
train_df['GarageQual'].fillna('NA',inplace=True)
train_df['GarageCond'].fillna('NA',inplace=True)
train_df[garage_columns].fillna('NA',inplace=True)
train_df[garage_columns].isna().any().sum()

0

In [26]:
train_df[garage_columns].isna().sum()

GarageType      0
GarageYrBlt     0
GarageFinish    0
GarageQual      0
GarageCond      0
dtype: int64

In [27]:
train_df['PoolQC'].fillna('NA', inplace=True)
train_df['Fence'].fillna('NA', inplace=True)
train_df['MiscFeature'].fillna('NA',inplace=True)

In [28]:
print(train_df.isnull().sum().any())
print(train_df.isna().sum().any())

False
False


In [32]:
# ALl null columns has been handled

_______________________________________________________________________________________
Now handle categorial columns
_______________________________________________________________________________________

In [29]:
#Encoding related to Basement Feature categories
BsmtQual_enc = {'Ex':7,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
BsmtCond_enc = {'Ex':7,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
BsmtExposure_enc = {'Gd':6,'Av':4,'Mn':3,'No':2,'NA':1}
BsmtFinType1_enc = {'GLQ':7,'ALQ':6,'BLQ':5,'Rec':4,'LwQ':3,'Unf':2,'NA':1}
BsmtFinType2_enc = {'GLQ':7,'ALQ':6,'BLQ':5,'Rec':4,'LwQ':3,'Unf':2,'NA':1}


In [30]:
#HeatingConditions Encoding
HeatingQC_enc = {'Ex':10,'Gd':4,'TA':3,'Fa':3,'Po':1}
Heating_enc = {'Floor':6,'GasA':5,'GasW':4,'Grav':3,'OthW':2,'Wall':1}
CentralAir_enc = {'Y':2,'N':1}

In [31]:
#Encoding For Kitchen Quality
KitchenQual_enc = {'Ex':10,'Gd':4,'TA':3,'Fa':2,'Po':1}

In [32]:
Functional_enc = {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':1,'Sal':2}

In [33]:
#FireplaceEncoding
FireplaceQu_enc = {'Ex':10,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}

In [34]:
#Garage Encoding
GarageType_enc = {'2Types':10,'Attchd':6,'Basment':5,'BuiltIn':4,'CarPort':3,'Detchd':2,'NA':1}
GarageFinish_enc = {'Fin':10,'RFn':5,'Unf':2,'NA':1}
GarageQual_enc = {'Ex':10,'Gd':6,'TA':5,'Fa':4,'Po':3,'NA':1}
GarageCond_enc = {'Ex':10,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}

In [35]:
#Pool Encoding
PoolQC_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'NA':1}
#External_condition
ExterQual_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'Po':1}
ExterCond_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'Po':1}

In [36]:
Street_enc = {'Grvl':1,'Pave':2}
Alley_enc = {'Grvl':2,'Pave':4,'NA':1}
LotShape_enc = {'Reg':6,'IR1':3, 'IR2':2,'IR3':1}
LandContour_enc = {'Lvl':6 ,'Bnk':3,'HLS':2,'Low':1}
Utilities_enc = {'AllPub':6,'NoSewr':3,'NoSeWa':2,'ELO':1}


In [37]:
LandSlope_enc = {'Gtl':4,'Mod':2,'Sev':1}
PavedDrive_enc = {'Y':4,'P':2,'N':1}

In [38]:
train_df['YrSold'].value_counts()

2009    338
2007    329
2006    314
2008    304
2010    175
Name: YrSold, dtype: int64

In [39]:
#Copy dataset to a new variable and then encode it
train_df_encoded = train_df.copy()

In [44]:
#BsmtQual_ce =  ce.OrdinalEncoder(mapping=[{'col':'BsmtQual','mapping':BsmtQual_enc}])
"""BsmtQual_ce =ce.OrdinalEncoder(cols=['BsmtQual'],return_df=True,
                               mapping=[{'col':'BsmtQual','mapping':BsmtCond_enc}])

train_df_encoded['BsmtQual'] = BsmtQual_ce.fit_transform(train_df_encoded)"""

"BsmtQual_ce =ce.OrdinalEncoder(cols=['BsmtQual'],return_df=True,\n                               mapping=[{'col':'BsmtQual','mapping':BsmtCond_enc}])\n\ntrain_df_encoded['BsmtQual'] = BsmtQual_ce.fit_transform(train_df_encoded)"

In [40]:
train_df.isnull().sum().any()

False

In [41]:
#Basement Features encoding
train_df_encoded['BsmtQual'] = train_df_encoded['BsmtQual'].map(BsmtQual_enc)
train_df_encoded['BsmtCond'] = train_df_encoded['BsmtCond'].map(BsmtCond_enc)
train_df_encoded['BsmtExposure'] = train_df_encoded['BsmtExposure'].map(BsmtExposure_enc)
train_df_encoded['BsmtFinType1'] = train_df_encoded['BsmtFinType1'].map(BsmtFinType1_enc)
train_df_encoded['BsmtFinType2'] = train_df_encoded['BsmtFinType2'].map(BsmtFinType2_enc)

#HeatingConditions Encoding
train_df_encoded['HeatingQC'] = train_df_encoded['HeatingQC'].map(HeatingQC_enc)
train_df_encoded['Heating'] = train_df_encoded['Heating'].map(Heating_enc)



In [42]:
train_df_encoded['CentralAir'] = train_df_encoded['CentralAir'].map(CentralAir_enc)
train_df_encoded['KitchenQual'] = train_df_encoded['KitchenQual'].map(KitchenQual_enc)
train_df_encoded['Functional'] = train_df_encoded['Functional'].map(Functional_enc)


train_df_encoded['FireplaceQu'] = train_df_encoded['FireplaceQu'].map(FireplaceQu_enc)

#Garage features encoding
train_df_encoded['GarageType'] = train_df_encoded['GarageType'].map(GarageType_enc)
train_df_encoded['GarageFinish'] = train_df_encoded['GarageFinish'].map(GarageFinish_enc)
train_df_encoded['GarageQual'] = train_df_encoded['GarageQual'].map(GarageQual_enc)
train_df_encoded['GarageCond'] = train_df_encoded['GarageCond'].map(GarageCond_enc)

In [43]:
train_df_encoded['PoolQC'] = train_df_encoded['PoolQC'].map(PoolQC_enc)
train_df_encoded['ExterQual'] = train_df_encoded['ExterQual'].map(ExterQual_enc)
train_df_encoded['ExterCond'] = train_df_encoded['ExterCond'].map(ExterCond_enc)

In [44]:
train_df_encoded['Street'] = train_df_encoded['Street'].map(Street_enc)
train_df_encoded['Alley'] = train_df_encoded['Alley'].map(Alley_enc)
train_df_encoded['LotShape'] = train_df_encoded['LotShape'].map(LotShape_enc)
train_df_encoded['LandContour'] = train_df_encoded['LandContour'].map(LandContour_enc)
train_df_encoded['Utilities'] = train_df_encoded['Utilities'].map(Utilities_enc)


In [45]:
train_df_encoded['LandSlope'] = train_df_encoded['LandSlope'].map(LandSlope_enc)

In [46]:
train_df_encoded['PavedDrive'] = train_df_encoded['PavedDrive'].map(PavedDrive_enc)

In [47]:
train_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   Alley          91 non-null     float64
 7   LotShape       1460 non-null   int64  
 8   LandContour    1460 non-null   int64  
 9   Utilities      1460 non-null   int64  
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   int64  
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [48]:
object_cols = train_df_encoded.select_dtypes('object')
object_cols

,MSZoning,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Electrical,GarageYrBlt,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Inside,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,SBrkr,2003.0,NA,NA,WD,Normal
1,RL,FR2,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,CBlock,SBrkr,1976.0,NA,NA,WD,Normal
2,RL,Inside,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,SBrkr,2001.0,NA,NA,WD,Normal
3,RL,Corner,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,BrkTil,SBrkr,1998.0,NA,NA,WD,Abnorml
4,RL,FR2,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,SBrkr,2000.0,NA,NA,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Inside,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,SBrkr,1999.0,NA,NA,WD,Normal
1456,RL,Inside,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,CBlock,SBrkr,1978.0,MnPrv,NA,WD,Normal
1457,RL,Inside,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,Stone,SBrkr,1941.0,GdPrv,Shed,WD,Normal
1458,RL,Inside,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,CBlock,FuseA,1950.0,NA,NA,WD,Normal


In [49]:
train_df_encoded.replace('NA','N',inplace=True)

In [55]:
#train_df_encoded['SaleType'] = train_df['SaleType']

In [50]:
train_df_encoded['SaleType'].shape

(1460,)

In [51]:
train_df_encoded['SaleType'].head(3)

0    WD
1    WD
2    WD
Name: SaleType, dtype: object

In [58]:
#train_df_encoded_enc = train_df_encoded['SaleType'].values.reshape(-1,1)

In [52]:
categorial_columns = train_df_encoded.select_dtypes('object').columns
categorial_columns

Index(['MSZoning', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'Electrical', 'GarageYrBlt',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')

In [53]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

#sale_type = train_df_encoded['SaleType'].reshape(-1,1)
#enc.fit(train_df_encoded['SaleType'].array.reshape(-1,1))
#enc.fit(train_df_encoded_enc)
#train_df_encoded['SaleType'] = enc.fit_transform(train_df_encoded_enc)
train_df_encoded[categorial_columns] = enc.fit_transform(train_df_encoded[categorial_columns].astype('str'))

In [54]:
#train_df_encoded['SaleType'] = enc.transform(train_df_encoded['SaleType'])
train_df_encoded[categorial_columns].head(3)

,MSZoning,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Electrical,GarageYrBlt,Fence,MiscFeature,SaleType,SaleCondition
0,3.0,4.0,5.0,2.0,2.0,0.0,5.0,1.0,1.0,12.0,13.0,1.0,2.0,4.0,89.0,4.0,1.0,8.0,4.0
1,3.0,2.0,24.0,1.0,2.0,0.0,2.0,1.0,1.0,8.0,8.0,2.0,1.0,4.0,62.0,4.0,1.0,8.0,4.0
2,3.0,4.0,5.0,2.0,2.0,0.0,5.0,1.0,1.0,12.0,13.0,1.0,2.0,4.0,87.0,4.0,1.0,8.0,4.0


In [62]:
#train_df_encoded['SaleType'].array.reshape(-1,1).shape

In [63]:
corr = train_df.corr()
corr

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,0.011156,-0.019761,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.051071,-0.005024,...,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917
MSSubClass,0.011156,1.000000,-0.215023,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.023573,-0.069836,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.019761,-0.215023,1.000000,0.100739,0.176561,-0.053457,0.036853,0.078686,0.105010,0.076670,...,-0.016780,0.069605,0.027366,0.023499,0.022969,0.114106,-0.059606,0.018942,-0.012094,0.209624
LotArea,-0.033226,-0.139781,0.100739,1.000000,0.105806,-0.005636,0.014228,0.013788,0.103321,0.214103,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,-0.028365,0.032628,0.176561,0.105806,1.000000,-0.091932,0.572323,0.550684,0.407252,0.239666,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,0.012609,-0.059316,-0.053457,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.125694,-0.046231,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,-0.012713,0.027850,0.036853,0.014228,0.572323,-0.375983,1.000000,0.592855,0.311600,0.249503,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,-0.021998,0.040581,0.078686,0.013788,0.550684,0.073741,0.592855,1.000000,0.176529,0.128451,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,-0.051071,0.023573,0.105010,0.103321,0.407252,-0.125694,0.311600,0.176529,1.000000,0.261256,...,0.159991,0.122528,-0.109907,0.019144,0.062248,0.011928,-0.029512,-0.006723,-0.008317,0.472614
BsmtFinSF1,-0.005024,-0.069836,0.076670,0.214103,0.239666,-0.046231,0.249503,0.128451,0.261256,1.000000,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420


In [55]:
def get_corelated_features(data,threshold,target):
    col_corr = set() #Set of all the names of corelated columns
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j]) > threshold:
                column_name = corr_matrix.columns[i]
                if column_name != target:
                    col_corr.add(column_name)
    return col_corr

In [56]:
corr_columns = get_corelated_features(train_df_encoded,0.8,'SalePrice')
corr_columns

{'1stFlrSF',
 'Exterior2nd',
 'FireplaceQu',
 'GarageArea',
 'GarageCond',
 'PoolQC',
 'TotRmsAbvGrd'}

In [66]:
#pip install -U scikit-learn

In [67]:
"""
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder


ordinal_features=['ExterQual','ExterCond','BsmtQual','BsmtCond',
'BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual',
'FireplaceQu','GarageQual','GarageCond','GarageFinish','Fence','PoolQC']

ordinal_orders=[ 
    #LotShape 
 
    #ExterQual 
['Fa','TA','Gd','Ex'], 
    #ExterCond
['Po','Fa','TA','Gd','Ex'],
    #BsmtQual 
['None','Fa','TA','Gd','Ex'], 
    #BsmtCond 
['None','Po','Fa','TA','Gd','Ex'], 
    #BsmtExposure 
['None','No','Mn','Av','Gd'], 
    #BsmtFinType1 
['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ],
    #BsmtFinType2 
['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ], 
    #HeatingQC 
['Po','Fa','TA','Gd','Ex'], 
    #KitchenQual 
['Fa','TA','Gd','Ex'],
    #FireplaceQu 
['None','Po','Fa','TA','Gd','Ex'], 
    #GarageQual 
['None','Po','Fa','TA','Gd','Ex'], 
    #GarageCond 
['None','Po','Fa','TA','Gd','Ex'], 
    #GarageFinish 
['None','Unf','RFn','Fin'], 
    #Fence 
['None','MnWw','GdWo','MnPrv','GdPrv'], 
    #PoolQC
['None','Fa','Gd','Ex']
]



transformer = make_column_transformer((OrdinalEncoder(categories= 'auto'),object_cols), remainder = "passthrough").fit(train_df_encoded)
"""

'\nfrom sklearn.compose import make_column_transformer\nfrom sklearn.preprocessing import OrdinalEncoder\n\n\nordinal_features=[\'ExterQual\',\'ExterCond\',\'BsmtQual\',\'BsmtCond\',\n\'BsmtExposure\',\'BsmtFinType1\',\'BsmtFinType2\',\'HeatingQC\',\'KitchenQual\',\n\'FireplaceQu\',\'GarageQual\',\'GarageCond\',\'GarageFinish\',\'Fence\',\'PoolQC\']\n\nordinal_orders=[ \n    #LotShape \n \n    #ExterQual \n[\'Fa\',\'TA\',\'Gd\',\'Ex\'], \n    #ExterCond\n[\'Po\',\'Fa\',\'TA\',\'Gd\',\'Ex\'],\n    #BsmtQual \n[\'None\',\'Fa\',\'TA\',\'Gd\',\'Ex\'], \n    #BsmtCond \n[\'None\',\'Po\',\'Fa\',\'TA\',\'Gd\',\'Ex\'], \n    #BsmtExposure \n[\'None\',\'No\',\'Mn\',\'Av\',\'Gd\'], \n    #BsmtFinType1 \n[\'None\',\'Unf\',\'LwQ\', \'Rec\',\'BLQ\',\'ALQ\' , \'GLQ\' ],\n    #BsmtFinType2 \n[\'None\',\'Unf\',\'LwQ\', \'Rec\',\'BLQ\',\'ALQ\' , \'GLQ\' ], \n    #HeatingQC \n[\'Po\',\'Fa\',\'TA\',\'Gd\',\'Ex\'], \n    #KitchenQual \n[\'Fa\',\'TA\',\'Gd\',\'Ex\'],\n    #FireplaceQu \n[\'None\',\'Po\',\'

In [ ]:
train_df_encoded['BsmtHalfBath'].replace(2,3,inplace=True)
train_df_encoded['BsmtHalfBath'].replace(1,2,inplace=True)
train_df_encoded['BsmtHalfBath'].replace(0,1,inplace=True)

In [89]:
train_df_encoded['BsmtFinSF1'].replace(0,1,inplace=True)
train_df_encoded['BsmtFinSF2'].replace(0,1,inplace=True)
train_df_encoded['TotalBsmtSF'].replace(0,1,inplace=True)
train_df_encoded['2ndFlrSF'].replace(0,1,inplace=True)

In [80]:
train_df_encoded['TotalBsmtSF'].value_counts()

1       37
864     35
672     17
912     15
1040    14
        ..
1838     1
1581     1
707      1
611      1
1542     1
Name: TotalBsmtSF, Length: 721, dtype: int64

In [83]:
train_df_encoded['OverallQual'] = train_df_encoded['OverallQual'] * 10
train_df_encoded['OverallCond'] = train_df_encoded['OverallCond'] * 10

In [102]:
#Group all relevant columns together
train_df_encoded['Bsmt'] = train_df_encoded['BsmtQual'] * train_df_encoded['BsmtCond'] * train_df_encoded['BsmtExposure'] * train_df_encoded['BsmtFinType1'] * train_df_encoded['TotalBsmtSF']  * train_df_encoded['BsmtFinType2']
train_df_encoded['Heat'] = train_df_encoded['HeatingQC'] * train_df_encoded['Heating']
train_df_encoded['Kitchen'] = train_df_encoded['CentralAir'] *train_df_encoded['KitchenQual'] * train_df_encoded['Functional']
train_df_encoded['Lot'] = train_df_encoded['LotFrontage'] * train_df_encoded['LotArea'] * train_df_encoded['Street'] * train_df_encoded['Alley'] * train_df_encoded['LotShape'] * train_df_encoded['LandContour'] * train_df_encoded['LandSlope']
train_df_encoded['ExtScore'] = train_df_encoded['ExterQual'] * train_df_encoded['ExterCond']
train_df_encoded['FireScore'] = train_df_encoded['Fireplaces'] * train_df_encoded['FireplaceQu']
train_df_encoded['Garage'] = train_df_encoded['GarageType'] *train_df_encoded['GarageFinish'] * train_df_encoded['GarageArea'] * train_df_encoded['GarageQual'] * train_df_encoded['GarageCond'] * train_df_encoded['PavedDrive']

In [135]:
columns_to_drop = ['LotFrontage','LotArea','Street','Alley' ,'LotShape','LandContour','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','TotalBsmtSF','Heating','HeatingQC','CentralAir','Fireplaces','FireplaceQu','GarageType','GarageFinish','GarageArea','GarageQual','GarageCond','PavedDrive',]

In [138]:

train_df_encoded.drop(columns=columns_to_drop,axis=1,inplace=True)

""
0
1
2
3
4
...
1455
1456
1457
1458


In [143]:
corr_columns = get_corelated_features(train_df_encoded,0.5,'SalePrice')
corr_columns

{'BedroomAbvGr',
 'BldgType',
 'Bsmt',
 'ExtScore',
 'Exterior2nd',
 'FireScore',
 'Foundation',
 'FullBath',
 'Garage',
 'GarageCars',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'Heat',
 'Kitchen',
 'KitchenQual',
 'PoolQC',
 'TotRmsAbvGrd',
 'YearBuilt',
 'YearRemodAdd'}

In [79]:
from sklearn.feature_selection import VarianceThreshold
var_thresh = VarianceThreshold(threshold=0.1)
zero_var_cols = var_thresh.fit_transform(train_df_encoded)
zero_var_cols

array([[1.00000e+00, 6.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 2.08500e+05],
       [2.00000e+00, 2.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 1.81500e+05],
       [3.00000e+00, 6.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 2.23500e+05],
       ...,
       [1.45800e+03, 7.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 2.66500e+05],
       [1.45900e+03, 2.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 1.42125e+05],
       [1.46000e+03, 2.00000e+01, 3.00000e+00, ..., 8.00000e+00,
        4.00000e+00, 1.47500e+05]])

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [57]:
train_df_encoded.shape

(1460, 81)

In [61]:
import xgboost as xgb

In [62]:
#pip install XGboost

In [63]:
X = train_df_encoded.drop(labels='SalePrice',axis=1)
y = train_df_encoded['SalePrice']

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [66]:
model.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=2,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [67]:
y_predicted = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
print("RMSE: %f" % (rmse))

RMSE: 78621.470522


In [68]:
from sklearn.model_selection import RandomizedSearchCV

In [69]:
booster = ['gbtree','gblinear']
n_estimator = [30,40,50,100,200,300]
max_depth = [2,3,5,10,15]
learning_rate = [0.1,0.15,0.2,0.3,0.5,0.7]
min_child_weight = [1,2,3,4]
base_score=[0.25,0.5,0.75,1]
aplha = [5,7,10,12,15]



In [70]:
hyperparameter_grid = {
    'n_estimator' : n_estimator,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'alpha' : aplha
}

In [71]:
grid_search_cv = RandomizedSearchCV(
        estimator=model,
        param_distributions=hyperparameter_grid,
        cv=5,n_iter=50,
        scoring = 'neg_mean_absolute_error',
        n_jobs=4,
        verbose=5,
        return_train_score=True,
        random_state=42
)

In [72]:
grid_search_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   10.6s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   19.5s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:   29.5s finished


[14:26:28] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(alpha=10, base_score=0.5,
                                          booster='gbtree', colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.3,
                                          enable_categorical=False, gamma=0,
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=5, min_child_weight=1,
                                          missing=nan,
                                          monotone_constrai...
                   iid='deprecated', n_iter=50, n_jobs=4,
                   param_distributions={'alpha': [5, 7, 10, 12, 15],
                                        'booster': ['gbtree', 

In [73]:
grid_search_cv.best_params_

{'n_estimator': 100,
 'min_child_weight': 3,
 'max_depth': 15,
 'learning_rate': 0.3,
 'booster': 'gbtree',
 'alpha': 15}

In [172]:
grid_search_cv.best_params_

{'n_estimator': 40,
 'min_child_weight': 2,
 'max_depth': 5,
 'learning_rate': 0.3,
 'booster': 'gbtree'}

In [74]:
model_tuned = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3,
                max_depth = 15, alpha = 15,n_estimator= 100,min_child_weight= 3,learning_rate= 0.3, booster= 'gbtree')

In [186]:
model_tuned_new = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3,
                max_depth = 15, alpha = 15,n_estimator= 100,min_child_weight= 3,learning_rate= 0.3, booster= 'gbtree')

In [75]:
model_tuned.fit(X_train,y_train)

y_tuned_pred = model_tuned.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_tuned_pred))
print("RMSE: %f" % (rmse))

[14:27:18] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


RMSE: 24928.464467


In [187]:
model_tuned_new.fit(X_train,y_train)

y_tuned_pred_new = model_tuned_new.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_tuned_pred_new))
print("RMSE: %f" % (rmse))

[18:17:26] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


RMSE: 25765.689221


In [85]:
def pre_process(train_df):
    train_df['LotFrontage'] = train_df['LotFrontage'].fillna(0)
    train_df['Alley'].fillna(0,inplace=True)
    train_df['MasVnrType'].fillna('None',inplace=True)
    train_df['MasVnrArea'].fillna(0,inplace=True)
    
    train_df['BsmtQual'].fillna('NA',inplace=True)
    train_df['BsmtCond'].fillna('NA',inplace=True)
    train_df['BsmtExposure'].fillna('NA',inplace=True)
    train_df['BsmtFinType1'].fillna('NA',inplace=True)
    train_df['BsmtFinType2'].fillna('NA',inplace=True)
    
    train_df[basement_cols].any().fillna('NA',inplace=True)
    train_df['Electrical'].fillna('SBrkr',inplace=True)
    train_df['FireplaceQu'].fillna('NA',inplace=True)
    
    garage_columns = ['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond' ]
    
    train_df['GarageType'].fillna('NA',inplace=True)
    train_df['GarageYrBlt'].fillna('NA',inplace=True)
    train_df['GarageFinish'].fillna('NA',inplace=True)
    train_df['GarageQual'].fillna('NA',inplace=True)
    train_df['GarageCond'].fillna('NA',inplace=True)
    train_df[garage_columns].fillna('NA',inplace=True)
    
    train_df['PoolQC'].fillna('NA', inplace=True)
    train_df['Fence'].fillna('NA', inplace=True)
    train_df['MiscFeature'].fillna('NA',inplace=True)
    
    #Encoding related to Basement Feature categories
    BsmtQual_enc = {'Ex':7,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
    BsmtCond_enc = {'Ex':7,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
    BsmtExposure_enc = {'Gd':6,'Av':4,'Mn':3,'No':2,'NA':1}
    BsmtFinType1_enc = {'GLQ':7,'ALQ':6,'BLQ':5,'Rec':4,'LwQ':3,'Unf':2,'NA':1}
    BsmtFinType2_enc = {'GLQ':7,'ALQ':6,'BLQ':5,'Rec':4,'LwQ':3,'Unf':2,'NA':1}

    
        #HeatingConditions Encoding
    HeatingQC_enc = {'Ex':10,'Gd':4,'TA':3,'Fa':3,'Po':1}
    Heating_enc = {'Floor':6,'GasA':5,'GasW':4,'Grav':3,'OthW':2,'Wall':1}
    CentralAir_enc = {'Y':2,'N':1}

    #Encoding For Kitchen Quality
    KitchenQual_enc = {'Ex':10,'Gd':4,'TA':3,'Fa':2,'Po':1}
    
    Functional_enc = {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':1,'Sal':2}
    
    #FireplaceEncoding
    FireplaceQu_enc = {'Ex':10,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
    
    #Garage Encoding
    GarageType_enc = {'2Types':10,'Attchd':6,'Basment':5,'BuiltIn':4,'CarPort':3,'Detchd':2,'NA':1}
    GarageFinish_enc = {'Fin':10,'RFn':5,'Unf':2,'NA':1}
    GarageQual_enc = {'Ex':10,'Gd':6,'TA':5,'Fa':4,'Po':3,'NA':1}
    GarageCond_enc = {'Ex':10,'Gd':5,'TA':4,'Fa':3,'Po':2,'NA':1}
    
    
        #Pool Encoding
    PoolQC_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'NA':1}
    #External_condition
    ExterQual_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'Po':1}
    ExterCond_enc = {'Ex':10,'Gd':5,'TA':3,'Fa':2,'Po':1}
    
    Street_enc = {'Grvl':1,'Pave':2}
    Alley_enc = {'Grvl':2,'Pave':4,'NA':1}
    LotShape_enc = {'Reg':6,'IR1':3, 'IR2':2,'IR3':1}
    LandContour_enc = {'Lvl':6 ,'Bnk':3,'HLS':2,'Low':1}
    Utilities_enc = {'AllPub':6,'NoSewr':3,'NoSeWa':2,'ELO':1}

    LandSlope_enc = {'Gtl':4,'Mod':2,'Sev':1}
    PavedDrive_enc = {'Y':4,'P':2,'N':1}
    
    #Copy dataset to a new variable and then encode it
    train_df_encoded = train_df.copy()
    
    #Basement Features encoding
    train_df_encoded['BsmtQual'] = train_df_encoded['BsmtQual'].map(BsmtQual_enc)
    train_df_encoded['BsmtCond'] = train_df_encoded['BsmtCond'].map(BsmtCond_enc)
    train_df_encoded['BsmtExposure'] = train_df_encoded['BsmtExposure'].map(BsmtExposure_enc)
    train_df_encoded['BsmtFinType1'] = train_df_encoded['BsmtFinType1'].map(BsmtFinType1_enc)
    train_df_encoded['BsmtFinType2'] = train_df_encoded['BsmtFinType2'].map(BsmtFinType2_enc)

    #HeatingConditions Encoding
    train_df_encoded['HeatingQC'] = train_df_encoded['HeatingQC'].map(HeatingQC_enc)
    train_df_encoded['Heating'] = train_df_encoded['Heating'].map(Heating_enc)
    
    
    train_df_encoded['CentralAir'] = train_df_encoded['CentralAir'].map(CentralAir_enc)
    train_df_encoded['KitchenQual'] = train_df_encoded['KitchenQual'].map(KitchenQual_enc)
    train_df_encoded['Functional'] = train_df_encoded['Functional'].map(Functional_enc)


    train_df_encoded['FireplaceQu'] = train_df_encoded['FireplaceQu'].map(FireplaceQu_enc)

    #Garage features encoding
    train_df_encoded['GarageType'] = train_df_encoded['GarageType'].map(GarageType_enc)
    train_df_encoded['GarageFinish'] = train_df_encoded['GarageFinish'].map(GarageFinish_enc)
    train_df_encoded['GarageQual'] = train_df_encoded['GarageQual'].map(GarageQual_enc)
    train_df_encoded['GarageCond'] = train_df_encoded['GarageCond'].map(GarageCond_enc)

    train_df_encoded['PoolQC'] = train_df_encoded['PoolQC'].map(PoolQC_enc)
    train_df_encoded['ExterQual'] = train_df_encoded['ExterQual'].map(ExterQual_enc)
    train_df_encoded['ExterCond'] = train_df_encoded['ExterCond'].map(ExterCond_enc)
    
    train_df_encoded['Street'] = train_df_encoded['Street'].map(Street_enc)
    train_df_encoded['Alley'] = train_df_encoded['Alley'].map(Alley_enc)
    train_df_encoded['LotShape'] = train_df_encoded['LotShape'].map(LotShape_enc)
    train_df_encoded['LandContour'] = train_df_encoded['LandContour'].map(LandContour_enc)
    train_df_encoded['Utilities'] = train_df_encoded['Utilities'].map(Utilities_enc)
    
    train_df_encoded['LandSlope'] = train_df_encoded['LandSlope'].map(LandSlope_enc)
    train_df_encoded['PavedDrive'] = train_df_encoded['PavedDrive'].map(PavedDrive_enc)
    
    object_cols = train_df_encoded.select_dtypes('object')
    train_df_encoded.replace('NA','N',inplace=True)
    categorial_columns = train_df_encoded.select_dtypes('object').columns
    
    from sklearn.preprocessing import OrdinalEncoder
    enc = OrdinalEncoder()
    train_df_encoded[categorial_columns] = enc.fit_transform(train_df_encoded[categorial_columns].astype('str'))
    return train_df_encoded
    
    


In [87]:
test_path = 'data/test.csv'
test_data = pd.read_csv(test_path)
test_data = pre_process(test_data)

In [88]:
test_data.shape

(1459, 80)

In [89]:
test_data.to_csv('data/test_converted.csv')

In [91]:
y_pred = model_tuned.predict(test_data)

In [92]:
y_pred.shape

(1459,)

In [93]:
my_submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': y_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('data/submission.csv', index=False)

In [94]:
def prepare_submission_file(data,path):
    my_submission = pd.DaraFrame({'Id': test_data.Id,'SalePrice': data})
    my_submission.to_csv(path,index=False)

In [95]:
from sklearn.feature_selection import mutual_info_regression

In [97]:
mutual_info = mutual_info_regression(X=X_train,y=y_train)
mutual_info

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
318,319,60,3.0,90.0,9900,2,NaN,6,1,6,...,0,0,1,4.0,1.0,0,4,2009,8.0,4.0
580,581,20,3.0,0.0,14585,2,NaN,3,6,6,...,0,0,1,4.0,1.0,0,6,2007,8.0,4.0
961,962,60,3.0,0.0,12227,2,NaN,3,6,6,...,0,0,1,4.0,1.0,0,7,2008,8.0,4.0
